In [1]:
import os
import difflib
import zipfile
import lxml.etree as etree
import random
import re
import sys
from datetime import datetime
import pandas as pd

import django
import platform

sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
import cities.models as cmodels
from django.contrib.auth.models import User

from aph_model_builder import *

/home/leey/venv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Slow version of gensim.models.doc2vec is being used
Slow version of Fasttext is being used


## Politician Parser

In [2]:
speakers = pd.read_csv('APH_leaders.csv')
#speakers = speakers.replace('NaN, np.nan')

In [3]:
speakers_past = speakers[:-1]
speakers_past

,Start_Date,End_Date,Last_Name,Title,Prefix,First_Name,Middle_Names,Additional
0,9.5.1901,23.11.1903,Holder,the Hon,Sir,Frederick,William,KCMG
1,2.3.1904,5.11.1906,Holder,the Hon,Sir,Frederick,William,KCMG
2,20.2.1907,23.7.1909,Holder,the Hon,Sir,Frederick,William,KCMG
3,28.7.1909,19.2.1910,Salmon,the Hon,NaN,Charles,Carty,NaN
4,1.7.1910,23.4.1913,McDonald,the Hon,NaN,Charles,NaN,NaN
5,9.7.1913,30.7.1914,Johnson,the Hon,NaN,William,Elliot,NaN
6,8.10.1914,26.3.1917,McDonald,the Hon,NaN,Charles,NaN,NaN
7,14.6.1917,3.11.1919,Johnson,the Hon,NaN,William,Elliot,(later Sir Elliot KCMG)
8,26.2.1920,6.11.1922,Johnson,the Hon,NaN,William,Elliot,(later Sir Elliot KCMG)
9,28.2.1923,3.10.1925,Watt,the Rt Hon,NaN,William,Alexander,NaN


In [6]:
df = speakers_past

In [25]:
entry = df.loc[5]
surname = entry['Last_Name']
first_name = entry['First_Name']
middle_name = entry['Middle_Names']

person = pm.Person.objects.filter(
surname = surname,
first_name = first_name,
information_source = "AustralianPoliticians"
)

person = person.filter(alt_first_names__contains=[middle_name])[0]

print(person.clean_name, person.pk, person.information_source)

William Johnson 15097 AustralianPoliticians


In [4]:
add_speaker(speakers_past)

In [1]:
# using iterrows()
def add_speaker(df):
    """Parses House Speakers from list to database"""

    for entry in df.iterrows():
        surname = entry['Last_Name']
        first_name = entry['First_Name']

        person = pm.Person.objects.get_or_create(
        surname = surname,
        first_name = first_name
        )
        #### Names
        #middle_names = []
        #for name in entry.Middle_Names:
        #    person.alt_first_names = middle_names.append(name)

        person.name_id = '10000'
        person.title = entry['Prefix']
        person.adel = entry['Title']
        
        #### Parliamentary periods
        person.positions = ['Speaker']

        person.save()

        post = pm.Post.objects.get_or_create(
        person = person,
        title = ['Speaker']
        )

        start_date = datetime.strptime(entry[Start_Date], '%d.%m.%Y')
        end_date = datetime.strptime(entry[End_Date], '%d.%m.%Y')

        post.start_date = start_date
        post.end_date = end_date
        post.save()


## Models Builder

### ParlPeriod

In [5]:
parlperiods = pd.read_csv('APH_parlperiod.csv')
parlperiods

,Parliament,Elected,Dissolved
0,First,30.3.1901,23.11.1903
1,Second,16.12.1903,5.11.1906
2,Third,12.12.1906,19.2.1910
3,Fourth,13.4.1910,23.4.1913
4,Fifth,31.5.1913,30.7.1914
5,Sixth,5.9.1914,26.3.1917
6,Seventh,5.5.1917,3.11.1919
7,Eighth,13.12.1919,6.11.1922
8,Ninth,16.12.1922,3.10.1925
9,Tenth,14.11.1925,9.10.1928


In [10]:
parlperiods_past = parlperiods[:-1]

In [11]:
add_parlperiod(parlperiods_past)